# Introduction to Natural Language Processing Catch-up 2

In [1]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt

from transformers import pipeline

from datasets import load_dataset
from datasets import load_metric

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification

## Library and dataset

In [4]:
'''Load IMDB dataset'''

imdb_db = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

1. Split the training set into a training and validation set

In [5]:
''' Making the split 80/20 for training and validation split'''

''' We set a seed to make the results reproductible '''
imdb_db_clean = imdb_db["train"].train_test_split(train_size=0.8, seed=42)
imdb_db_clean["validation"] = imdb_db_clean.pop("test")
imdb_db_clean["test"] = imdb_db["test"]
imdb_db_clean

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

2. Make sure the training and validation set have the same proportion of both class.

In [6]:
''' Check if both validation and train split have the same proportion of label '''

validation_labels = imdb_db_clean["validation"]["label"]
train_labels = imdb_db_clean["train"]["label"]
print(train_labels.count(0) / train_labels.count(1))
print(validation_labels.count(0)/ validation_labels.count(1))

1.0012007204322593
0.9952114924181963


## Fine-tuning a model

1. Fine-tune the model on the training data. 

In [7]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = imdb_db_clean.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Parameter 'function'=<function tokenize_function at 0x7f1223779f80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets.remove_columns("text")

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [9]:
training_args = TrainingArguments("test-trainer", num_train_epochs=1)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to("cuda")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [12]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Step,Training Loss
500,0.407200
1000,0.346300
1500,0.318900
2000,0.274700
2500,0.262800


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=2500, training_loss=0.3219896484375, metrics={'train_runtime': 1868.256, 'train_samples_per_second': 10.705, 'train_steps_per_second': 1.338, 'total_flos': 4962443059609440.0, 'train_loss': 0.3219896484375, 'epoch': 1.0})

2. Evaluate the model in term of accuracy on the test data.

In [13]:
''' Evaluation sur le dataset de validation '''
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 8


(5000, 2) (5000,)


In [15]:
preds = numpy.argmax(predictions.predictions, axis=-1)

metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9252, 'f1': 0.9252299080367854}

In [16]:
''' Evaluation sur le split de test '''

predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25000
  Batch size = 8


(25000, 2) (25000,)


In [17]:
preds = numpy.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.93124, 'f1': 0.9313525817659041}

On obtient une accuracy de 93% ce qui est nettement mieux que les résultat de notre Naive Bayes Calsifier.

3. For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [22]:
wrong = []
for pred, label, text in zip(preds,imdb_db["test"]["label"], imdb_db["test"]["text"]):
  if (pred != label):
    wrong.append((pred,label,text))

wrong[:2]

[(1,
  0,
  "First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!"),
 (1,
  0,
  "Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an ass

Le premier commentaire mal classifié est le même que dans notre précedente implémentation d'un classifier, l'avis est partagé entre le fait que ça soit un film "bon" pour un film d'action mais qu'il n'est pas révolutionnaire dans sa manière d'aborder le genre. Il semble toujour étrange que cet avis soit considéré dans le dataset comme un avis négatif.

Pour le second commentaire l'avis s'attarde sur un résumé des personnages et s'attache particulèrement sur le personnage qui selon l'auteur est le plus apréciable du film. Comme l'avis précédent ici l'auteur est partagé dans son avis mais trouv tout de meme le film "correct" et regardable qui fait que notre modèle le considère comme un avis positif.

4. What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course?

LEs avantages d'utiliser ce modèle sont que l'on a des résultat bien plus précis que lorsqu'on utilise un Naive Bayes model, on peut plus facilement process beaucoup plus de données et adapter le modèle à notre dataset poru qu'il soit spécialisé.
Les incovénient sont que le modèle requiert une grande capacité de calcul pour être entrainé et fine-tuned mais on peut avoir plus facilement des bias dans notre apprentissage.

## Theoritical questions 

1. What is the purpose of subword tokenization used by transformer models? 

L'avantage de la *subword tokenization* est de donner un place plus facilement aux mots communs dans notre vocabulaire et de réduire grandement la taille du vocabulaire en fiasant un mélange entre la tokenization de mot et de caractères.
Cette méthode réduit grandement la taille du vocabulaire qui est plus facile à manipuler.
Les nouveaux mots sont associés à l'aide de bouts de mots ou des caractères qui sont déjà présents dans le vocabulaire.

2. What are the differences between an RNN and an LSTM? 

La principale différence entre un RNN et un LSTM c'est la compléxité des neurones du LSTM. En effet dans un LSTM les neurones sont composés de portes qui permettent de réguler la propagation de l'information à travers les neuronnes, une porte pour l'entrée, la sortie et une pour ce qui doit etre ignoré. Deplus les neurones des LSTM ont des connexions cycliques au sein du réseau. Un LSTM es designer pour résoudre le problème dont les RNN souffrent, le *vanishing gradient problem* dans lequel la descente de gradient est trop faible pour faire changer les poids du RNN et ducoup sont ignorés.

3. When building an encoder-decoder model using an RNN, what is the purpose of adding attention? 

Le problème du modèle *encoder-decoder* avec un RNN c'est que le modèle à du mal lorsqu'il s'aggit de décoder des longues séquences.
L'alignement résoud ce problème en faisant de l'alignement et de la tradution des séquences en se focalisant uniquement sur les parties les plus pertinentes.
Pour faire cela au lieu de donner en entrer un simple vecteur on fourni un vecteur de context au modèle pour qu'il puisse plus facilement gérer les longues séquence à encoder et decoder.

4. In a transformer model what is the multihead attention used for? 

La *Self-Attention* a pour but de faire interagir chaque input avec les autres et de mettre en valeur les autres inputs avec lesquels il est intéressant d'interagir.
Le but du *multihead attention* est de calculer en parallele une attention pour l'encoder, le décodeur et des les combiner ensemble pour former une attention encodeur-décodeur. L'interêt de faire ca en parallèle est de detecter des relations différentes et d'encoder plus facilement toutes les nuances d'un même mot.

5. In a transformer, what is the purpose of positional embedding? 

Le but du *positional embeding* est de remplacer les index numériques pour donner la position des éléments dans une séquence. Le problème des index est que si on commence à avoir de longues séquences les index vont commencer à devenir énormes, et si on essaie de normaliser ces index on va avoir des conflits entre les différentes séquences où des mêmes index n'indiquent pas les même positions. C'est pour cela que le *positional embeding* encode les postions dans des vecteurs et stocke tout dans une grande matrice.

6. What is the purpose of having benchmarks to evalute models?

Le problème d'évaluer un modèle de NLP est que les méthodes classiques pour évauler un modèle requierent une fonction de perte ou une fonction de claire, ce qui est compliqué à mettre en place de manière globale sur le NLP qui est un problème large qui va de la classification à la génération de réponse. Deplus la notion de *ground truth* est assez flou dans les problèmes des générations de text ou il n'y a pas une seule solution correcte. c'est pourquoi on utilise des Benchmarks pre-etabli pour evaluer les modèls de manière standardisée.

7. In the BERT model, describe the two tasks used for pre-training (unsupervised) with a few sentences. 

Masked Language Modeling:
Le but de ce modèle est de masquer un mot dans une phrase et essayer à l'aide du context donné par les mots à droite et à gauche de retrouver le mot qui correspond le mieux.

Next Sentence Prediction:
Le but de ce modèle est en partant de deux phrases d'entrée de determiner si la 2eme est celle qui suit la 1ere. Pour cela le modèle se sert des tokens qui séparent chaque phrases ainsi que du *positional embeding* des phrases.


8. In a few sentences, explain how the triplet loss is used to train a bi-encoder model for semantic similarity? 

La *triplet loss* se sert d'un triplet ancre, positif et négatif pour permettre de plus facilement lier semantiquement les mots. Le but est de faire coincider les plus possible les valeurs du positif avec celle de l'ancre tout en gardant les nagatifs en tant qu'opposé ou du moins distinct de lancre. Pour cela on se sert du sentence embeding du bi-encoder pour lier les phrases avec leur ancre.

9. What is the purpose of using an Approximate Nearest Neighbour method to speed up search? 

Le but d'utiliser une mathode de *Approximate Nearest Neighbour*  pour accelerer la recherche est de convertir les mots et les text sous forme de vecteurs et de trouver des vecteurs similaires qui vont accelerer les débuts de nos méthodes pour associer des mots ensembles. 

In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
